In [33]:
from brt.router import FusedRandomGatherRouter, FusedRandomScatterRouter
import brt.nn as nn
import brt
import torch


@brt.domain
class FusedMoE(nn.Module):
    def __init__(self, route_num):
        super().__init__()
        self.scatter_router = FusedRandomScatterRouter(
            route_num=route_num, supported_capacities=[2, 4, 8, 16]
        )
        self.gather_router = FusedRandomGatherRouter(route_num=route_num)

    def forward(self, inputs):
        route_results, reverse_indices, _ = self.scatter_router(inputs)
        route_results = self.gather_router(route_results, reverse_indices)
        return route_results


fused_moe = FusedMoE(route_num=2)
input_tensor = torch.rand((10, 10))
output_tensor = fused_moe(input_tensor)
print(torch.isclose(output_tensor, input_tensor))



tensor([[True, True, True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True, True, True]])
